In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install import-ipynb
import import_ipynb

In [ ]:
%cd drive
%cd 'My Drive'
%cd 'RL'
%cd 'Homework'
%cd 'Homework 4'

In [3]:
'''Needed Libraries and Environment'''
import Environment
import Modified_Environment
import matplotlib.pyplot as plt
import numpy as np
import math
import random
from collections import defaultdict
import sys
import itertools
import matplotlib
import pandas as pd
import pickle


'''Constants Defined'''
STATE = 0
ACTION = 1
REWARD = 2
MAX_T = 10000
TOTAL_EPISODES = 2000
gamma = 0.8
ALPHA = 0.2
MIN = 0.001
DECAY_FACTOR = 0.99

importing Jupyter notebook from Environment.ipynb
importing Jupyter notebook from Modified_Environment.ipynb


In [28]:
class OffPolicyMCAgent():
  
    def __init__(self, env, behaviour_policy, gamma):

      self.env = env
      self.Q = defaultdict(lambda: np.zeros(env.action_space.n))
      self.C = defaultdict(lambda: np.zeros(env.action_space.n))
      self.actions = env.action_space.n
      self.behaviour_policy = behaviour_policy
      self.gamma = gamma
      self.G = 0.0
      self.W = 1.0
      self.Off_MC_episode_scores = []
      self.returns_sum = defaultdict(float)
      self.returns_count = defaultdict(float)
      self.episode_information = []
      self.states_in_episode = []

    def target_policy(self, state):
        A = np.zeros_like(self.Q[state], dtype=float)
        best_action = np.argmax(self.Q[state])
        A[best_action] = 1.0
        return A


    def reach_island(self):
        #epsilon = 1 #<-- uncomment if you want to use decaying epsilon 
        #epsilon = 0.3 #<-- uncomment if you want to use constant epsilon 
        for episode in range(1, TOTAL_EPISODES + 1):
            episode_score = 0
            if episode % 4 == 0:
                print("\rEpisode {}/{}.".format(episode, TOTAL_EPISODES), end="")
                sys.stdout.flush()
                epsilon = max(epsilon * DECAY_FACTOR, MIN) #<-- uncomment if you want to use decaying epsilon 
            done = False
            state = self.env.reset()
            for t in itertools.count():
                probs = self.behaviour_policy(self.Q, state, epsilon)
                action = np.random.choice(np.arange(len(probs)), p = probs)
                state_prime, reward, done, _ = self.env.step(action)
                episode_score += reward
                self.episode_information.append((state, action, reward))
                self.states_in_episode.append(state)
                if done or t > MAX_T :
                  break
                state = state_prime
            self.off_policy_returns(epsilon)
            self.Off_MC_episode_scores.insert(0, episode_score)


    def off_policy_returns(self, epsilon):
        self.G, self.W = 0, 1
        for ep in range(len(self.episode_information))[::-1]:
            state, action, reward = self.episode_information[ep]
            self.G = self.gamma*self.G + reward
            self.C[state][action] += self.W
            self.Q[state][action] += (self.W/self.C[state][action]) * (self.G - self.Q[state][action])
            if action != np.argmax(self.target_policy(state)):
                break
            self.W = self.W * (self.target_policy(state)[action]/self.behaviour_policy(self.Q, state, epsilon)[action])


In [5]:
def create_behaviour_policy(nA):
    def policy_fn(Q, observation, epsilon):
        A = np.ones(nA, dtype=float) * epsilon / nA
        best_action = np.argmax(Q[observation])
        A[best_action] += (1.0 - epsilon)
        return A
    return policy_fn

In [29]:
# env =  Environment.GridworldEnv() #<--uncomment for normal environment
#env =  Modified_Environment.GridworldEnv() # <--uncomment for modified environment (bonus part)
behaviour_policy = create_behaviour_policy(env.action_space.n)
agent = OffPolicyMCAgent(env, behaviour_policy, gamma)
agent.reach_island()


Episode 2000/2000.

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(range(TOTAL_EPISODES), agent.Off_MC_episode_scores, color = '#633974', label = '$\epsilon$ = from $1$ to $0.01$')
plt.xlabel('episodes ->')
plt.ylabel('epsiode score ->')
plt.title('Off-policy Monte Carlo')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
window_size = 50
smoothed_score = pd.Series(agent.Off_MC_episode_scores).rolling(window_size , min_periods = window_size).mean()
plt.plot(smoothed_score, color = '#633974', label = '$\epsilon$ = from $1$ to $0.01$')
#plt.annotate(smoothed_score.iloc[-1], xy=(2000, smoothed_score.iloc[-1]), xytext=(2000, smoothed_score.iloc[-1]), color = '#633974',) # <--uncomment for modified environment (bonus part)
plt.xlabel("epsiode ->")
plt.ylabel("epsiode score (smoothed) -> ")
plt.title('Off-policy Monte Carlo')
plt.legend()
plt.show()

In [32]:
#i wrote my reward data into a .txt file to use it later.

# with open('Off_MC_bonus_scores', 'wb') as fp:
    # pickle.dump(agent.Off_MC_episode_scores, fp)